In [ ]:
import pandas as pd
from functools import reduce
import random

In [ ]:
file_paths = ['../data/cleanedData/oxygenAB.parquet',
              '../data/cleanedData/nitrate.parquet',
              '../data/cleanedData/phosphate.parquet',
              '../data/cleanedData/ammonium.parquet']

dfs = [pd.read_parquet(path) for path in file_paths]

merged = reduce(lambda left, right: pd.merge(left, right, how='inner', on='datumEindeMeting'), dfs)
# 1.4 is the std of the phosphate
phosphate_average = merged['phosphateValue'].mean() + random.uniform(1.4,-1.4)
merged['phosphateValue'] = merged['phosphateValue'].map(lambda phosphate: phosphate_average if pd.isnull(phosphate) else phosphate)
merged.to_parquet('../data/cleanedData/allData.parquet')

In [ ]:
rain = pd.read_csv("../data/cleanedData/precipitation.csv")

In [ ]:
rain = rain.rename(columns={'Date': 'datumEindeMeting'})
rain.info()


In [ ]:
merged.info()

In [ ]:
rain["datumEindeMeting"].dtype

In [ ]:
rain = rain.astype({"datumEindeMeting": "datetime64[ns]"})
date_range = pd.date_range(start=rain["datumEindeMeting"].min(), end=rain["datumEindeMeting"].max(), freq="T")
minute_df = pd.DataFrame({"datumEindeMeting": date_range})  

def assign_precipitation(row):
    date = row["datumEindeMeting"].date()
    daily_row = rain.loc[rain["datumEindeMeting"].dt.date == date]
    if not daily_row.empty:
        return daily_row["Rain (mm)"].values[0]
    else:
        return 0.0

minute_df["precipitation"] = minute_df.apply(assign_precipitation, axis=1)
precip_resampled = rain.set_index("datumEindeMeting")["Rain (mm)"].resample('T').ffill()
precip_resampled_df = precip_resampled.to_frame("rain_per_minute").reset_index()  
merged_df = minute_df.merge(precip_resampled_df, how="left", on="datumEindeMeting")
merged = merged.merge(merged_df[['datumEindeMeting', 'precipitation']], how='left', on='datumEindeMeting')


In [ ]:
merged